# Compute portfolio average short-term rating
The following case-study demonstrates how to compute the average short-term rating of
your portfolio.
Before we start looking at a concrete example, let's recap the steps that are
necessary to compute the average short-term portfolio rating.

Translating short-term ratings into numerical rating scores and vice versa isn't as
straightforward as with long-term ratings (see
[Short-term ratings](/../short_term_ratings/)).

The below chart shows the steps to compute the average portfolio rating for a
two-security portfolio.

!["Two-security example using average equivalent value (AEV)"](/../media/flow_chart_example_AEV.png "Two-security example using average equivalent value (AEV)")

- Step 1:<br>
  Translate all short-term ratings from available rating agencies into an average
  equivalent long-term rating score or into a so-called **average equivalent value
  (AEV)**.
- Step 2:<br>
  Consolidate all ratings on a security basis into one equivalent long-term rating
  score.<br>
  For example, you could decide to choose the worst rating (read: highest
  numerical score) among all rating agencies.<br>
  As a result, you'll end up with the worst (i.e. highest) AEV per security.
- Step 3:<br>
  Compute the weighted AEV for the portfolio. That is, multiply an individual
  security's weight with its worst AEV. You'll end up with this security's
  contribution to the portfolio AEV.<br>
  Finally, sum up the individual securities' contributions.<br>
  Essentially, this is simply computing the sum-product of the weights and the worst
  AEVs.
- Step 4:<br>
  Translate the portfolio AEV back into a human-readable short-term rating, while
  using the agency's rating scale of your choice.


## Preliminary tasks
As a first step, we are going to import a portfolio into a `pd.DataFrame`. We'll call
it `port_df`. This dataframe comprises a number of securities with respective weights
 and short-term ratings.

In [ ]:
import pandas as pd

import pyratings as rtg

port_df = pd.read_excel(
    "portfolio.xlsx",
    sheet_name="short_term_ratings",
)

port_df

## Clean ratings
If you take a closer look at row 5 and 13, you see that there are two ratings from
Moody's which are based on public information (indicated by "(P)").
We need to get rid of this prefix, otherwise the rating can't be correctly identified
 by **pyratings**.

In [ ]:
port_df_clean = rtg.get_pure_ratings(
    ratings=port_df.loc[:, ["Moody", "SP", "Fitch", "DBRS"]]
)
port_df_clean

## Consolidate ratings (step 1 and 2)
Next, we are consolidating the clean ratings. We take the conservative approach and
identify the worst numerical rating score assigned to every individual security.

We can use the
[``get_worst_scores``](../../api_reference/#pyratings.consolidate.get_worst_scores)
function. **pyratings** will automatically convert the short-term ratings into AEVs
(step 1) and identify the hightest number (read: worst rating score) (step 2).

In [ ]:
port_worst_scores_df = rtg.get_worst_scores(
    ratings=port_df_clean,
    rating_provider_input=["Moody", "SP", "Fitch", "DBRS"],
    tenor="short-term",
)

port_worst_scores_df = pd.concat(
    [
        port_df.loc[:, ["ISIN", "weight"]],
        port_worst_scores_df,
    ],
    axis=1
)

port_worst_scores_df

## Compute the weighted AEV (step 3)
Now, we need to compute a weighted average rating score. We use the
[``get_weighted_average``](../../api_reference/#pyratings.aggregate.get_weighted_average)
function.

In [ ]:
avg_rtg_score = rtg.get_weighted_average(
    data=port_worst_scores_df["worst_scores"],
    weights=port_worst_scores_df["weight"] / 100,
)

print(f"Average rating score: {avg_rtg_score:.2f}")
print(f"Average rating score (rounded): {round(avg_rtg_score):.2f}")

## Going back to human-readable ratings (step 4)
Translating rating scores into short-term ratings is somewhat different compared to
long-term ratings
(see [Short-term ratings](../../short_term_ratings/#theres-one-more-catch)).
We need to decide which strategy we will use: ``best``, ``base``, or ``worst``.
Choosing the strategy influences the outcome. The rounded rating score is 7.

We are going to use the
[``get_ratings_from_scores``](../../api_reference/#pyratings.get_ratings.get_ratings_from_scores)
function.
The input parameter ``short_term_strategy`` defines the strategy, which will be used
to translate AEVs back into short-term ratings.

The following table describes how this rating score will be translated into a
short-term rating depending on the strategy.

![Translation short-term ratings into long-term rating scores with all strategies](/.
./media/translation_st_ratings_lt_rating_scores_all_strats.png)

In [ ]:
avg_rtg_best = rtg.get_ratings_from_scores(
    rating_scores=avg_rtg_score,
    rating_provider="S&P",
    tenor="short-term",
    short_term_strategy="best",
)

avg_rtg_base = rtg.get_ratings_from_scores(
    rating_scores=avg_rtg_score,
    rating_provider="S&P",
    tenor="short-term",
    short_term_strategy="base",
)

avg_rtg_worst = rtg.get_ratings_from_scores(
    rating_scores=avg_rtg_score,
    rating_provider="S&P",
    tenor="short-term",
    short_term_strategy="worst",
)

print(f"Average portfolio rating (strategy: 'best'): {avg_rtg_best}")
print(f"Average portfolio rating (strategy: 'base'): {avg_rtg_base}")
print(f"Average portfolio rating (strategy: 'worst'): {avg_rtg_worst}")

There you go. The average rating of the portfolio is either A-1 (``best``) or A-2
(``base`` and ``worst``).